   This notebook shows an example of the k-NN algorithm applied to classification of time series data (accelerometer data). We run the knn algorithm with the raw signals and we compare two similarity measures: euclidian distance and Dynamic Time Warping distance that takes into account a lack of synchornisation between two time series.

## Imports

In [31]:
import os
import numpy as np
from scipy.stats import mode
from sklearn.metrics import classification_report, f1_score
from sklearn import preprocessing
import time
import matplotlib.pylab as plt 
%matplotlib

Using matplotlib backend: MacOSX


## Loading Data

We load the raw accelerometer signals of the first axis x. We cannot use the other axis as the knn algorithm finds the k closest neighbors of a sample according to a distance and it does not make sense to compare x axis signals to y axis signals. Moreover we do not consider the entire dataset (7352 training samples and 2947 test samples) since Dynamic Time Warping Distance becomes extremely time consumming when the number of samples to compare grows.  
Each sample of the data set is a 2.56s window of an activity being performed recorded at a 50Hz rate which makes 128 readings per sample.

In [32]:
os.chdir('../data')

n_skip = 20 # We select a substet of the training and test set (one every 20 sample)...

X_train = np.loadtxt('X_train.txt')[::n_skip]
X_test = np.loadtxt('X_test.txt')[::n_skip]

print("X_train shape : {}".format(X_train.shape))
print("X_test shape : {}".format(X_test.shape))

X_train shape : (368, 128)
X_test shape : (148, 128)


Now we load the label vectors.

In [33]:
y_train = np.loadtxt('y_train.txt')[::n_skip]
y_test = np.loadtxt('y_test.txt')[::n_skip]

print("y_train shape : {}".format(y_train.shape))
print("y_test shape : {}".format(y_test.shape))

label_names = ['Walking', 'Walking upstairs', 'Walking downstairs', 'Sitting', 'Standing', 'Laying']

y_train shape : (368,)
y_test shape : (148,)


## Dynamic Time Warping and Euclidian distances

 Two functions that computes euclidian distance and Dynamic Time Warping distance between two time series.

In [34]:
def euclidian_distance(x1,x2):
    return np.linalg.norm(x1-x2)

 Dynamic Time Warping is a distance measure between tow time series taking into account the fact one might accelerate or decelarate.

In [35]:
def DTW_distance(x1, x2, w=1000, distance=lambda x1,x2 : abs(x1-x2)):
    """Computes Dynamic Time Wrapping distance between time series x1 and x2
    INPUTS:
        -x1 is a (n,) numpy array
        -x2 is a (m,) numpy array
        -w is the DTW window (type int)
    OUTPUTS:
        - the DTW distance between x1 and x2 (float)
    """
    # time series lengths
    n = x1.shape[0]
    m = x2.shape[0]
    w = max(w, abs(n-m)) 

    # Initialiaze distance matrix
    DTW = np.zeros((n,m)) 
    DTW[0,0] = distance(x1[0],x2[0])
    for i in range(1,n):
        DTW[i,0] = DTW[i-1,0] + distance(x1[i],x2[0])
    for j in range(1,m):
        DTW[0,j] = DTW[0,j-1] + distance(x1[0],x2[j])

    # We fill the rest of the distance matrix
    for i in range(1,n):
        for j in range(int(max(1,i-w)),min(m,i+w)):
            dist = distance(x1[i],x2[j])
            DTW[i,j] = dist + min(DTW[i-1,j], DTW[i,j-1], DTW[i-1,j-1])
           
    return DTW[-1,-1]

### K-NN algorithm 

Brute force knn algorithm.

In [36]:
def make_distance_matrix(X_train, X_test, w=60, distance = euclidian_distance):
    """ This function returns the distance matrix between samples of X_train and X_tes according to a 
    similarity measure.
    INPUTS:
        - X_train a (n, p) numpy array with n:number of training samples and m: number of features
        - X_test a (m, p) numpy array with m: number of test samples and m as above
        - w DTW window
        - distance_type the type of distance to consider for the algorithm ['euclidian', 'DTW']
    OUTPUTS:
        - dis_m a (m,n) numpy array with dist_m[i,j] = distance(X_test[i,:], X_train[j,:])
    """
    
    # Distance matrix calculation
    n = X_train.shape[0]
    m = X_test.shape[0]  
    dist_m = np.zeros((m,n))
    for row, test_spl in enumerate(X_test):
        for col, train_spl in enumerate(X_train):
            if distance == euclidian_distance:
                dist_row_col = distance(test_spl, train_spl)
                dist_m[row,col] = dist_row_col
            else:
                dist_row_col = distance(test_spl, train_spl, w)
                dist_m[row,col] = dist_row_col                    
    return dist_m

In [37]:
def find_k_closest(dist_m, y_train, k):
    """ This function returns the most represented label among the k nearest neighbors of each sample from
    X_test.
    INPUTS:
        - dist_m a (m,n) numpy array with dist_m[i,j] = distance(X_test[i,:], X_train[j,:])
        - y_train a (n,) numpy array with X_train labels
        - k number of neighbors to consider (int)
    OUPUTS:
        - y_pred a (m,) numpy array of predicted labels for X_test
    """
    knn_indexes = np.argsort(dist_m)[:,:k]
    knn_labels = y_train[knn_indexes]
    y_pred = mode(knn_labels, axis=1)[0]
    return y_pred

## kNN with euclidian distance

We first compare the time series based on the euclidian distance between them.

In [38]:
start = time.time()
dist_m = make_distance_matrix(X_train, X_test)
stop = time.time()

print("Execution time: {:.2f} min {:.2f} s ".format((stop-start) // 60, (stop-start) % 60))

Execution time: 0.00 min 0.37 s 


In [39]:
k = 1
y_pred = find_k_closest(dist_m, y_train, k)

print("Parameters:")
print("k = {}".format(k))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 1


Test set report
                    precision    recall  f1-score   support

           Walking       0.50      0.48      0.49        23
  Walking upstairs       0.52      0.68      0.59        25
Walking downstairs       0.88      0.30      0.45        23
           Sitting       0.43      0.45      0.44        22
          Standing       0.41      0.56      0.47        27
            Laying       0.44      0.39      0.42        28

       avg / total       0.52      0.48      0.48       148



In [40]:
def find_k_best(dist_m, y_train, y_test, k_range=np.arange(1,22)):
    k_range = np.arange(1,22) # range of k to test
    f1_scores = np.empty(k_range.shape) # we are going to store f1 scores here
    # now we loop over k_range and compute f1_scores...
    for k in k_range:
        y_pred = find_k_closest(dist_m, y_train, k=k)
        f1_scores[k-1] = f1_score(y_test, y_pred, average='macro')
    return k_range[np.argmax(f1_scores)]

In [41]:
k = find_k_best(dist_m, y_train, y_test, k_range=np.arange(1,22))
y_pred = find_k_closest(dist_m, y_train, k)

print("Parameters:")
print("k = {}".format(k))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 1


Test set report
                    precision    recall  f1-score   support

           Walking       0.50      0.48      0.49        23
  Walking upstairs       0.52      0.68      0.59        25
Walking downstairs       0.88      0.30      0.45        23
           Sitting       0.43      0.45      0.44        22
          Standing       0.41      0.56      0.47        27
            Laying       0.44      0.39      0.42        28

       avg / total       0.52      0.48      0.48       148



//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## kNN with Dynamic Time Warping distance

Now we use Dynamic Time Warping Distance (DTW) to compare time series. The results are better with this measure due to the nature of the dataset.  

First, as explained in the intro, the samples are 2.56s windows of an activity being performed and these windows overlap so the samples do not share a common time stamp. Secondly, if we compare two people starting walking, their paces may be different and their accelerometer recordings are probably not going to be synchornized.
  
  Dynamic Time Warping can overcome this issue of synchronization by matching points that are not facing each other.
  
  Now if we compare to the previous cell, the results are around 10% better with DTW but the execution time is almost 24 min while it was under a second with euclidian distance.

In [43]:
w = 1000

start = time.time()
dist_m_dtw = make_distance_matrix(X_train, X_test, distance=DTW_distance, w=w)
stop = time.time()

print("Execution time: {:.2f} min {:.2f} s ".format((stop-start) // 60, (stop-start) % 60))

Execution time: 22.00 min 23.73 s 


In [27]:
k = 1
y_pred = find_k_closest(dist_m_dtw, y_train, k=k)

print("Parameters:")
print("k = {}".format(k))
print("w = {}".format(w))
print("\n")
print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 1
w = 1000


Test set report
                    precision    recall  f1-score   support

           Walking       0.79      0.83      0.81        23
  Walking upstairs       0.61      0.88      0.72        25
Walking downstairs       1.00      0.48      0.65        23
           Sitting       0.53      0.41      0.46        22
          Standing       0.44      0.59      0.51        27
            Laying       0.50      0.43      0.46        28

       avg / total       0.64      0.60      0.60       148



In [30]:
k = find_k_best(dist_m_dtw, y_train, y_test, k_range=np.arange(1,22))
y_pred = find_k_closest(dist_m, y_train, k)

print("Parameters:")
print("k = {}".format(k))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 1


Test set report
                    precision    recall  f1-score   support

           Walking       0.79      0.83      0.81        23
  Walking upstairs       0.61      0.88      0.72        25
Walking downstairs       1.00      0.48      0.65        23
           Sitting       0.53      0.41      0.46        22
          Standing       0.44      0.59      0.51        27
            Laying       0.50      0.43      0.46        28

       avg / total       0.64      0.60      0.60       148

